In [2]:
import pandas as pd
from prophet import Prophet

Install necessary packages, namely pandas and prophet if you do not have already

In [5]:
df = pd.read_csv('GNPDataset.csv', on_bad_lines='skip')

In [6]:
df.head()

,Year,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,AnnualTotal,Textbox4
0,2021,"23,417","17,238","30,976","53,134","175,277","587,237","796,868","670,628","533,388","149,564","26,300","17,629","3,081,656",81.40%
1,2020,"13,651","18,020","24,377",0,0,"202,701","453,977","459,121","343,911","125,544","29,366","28,196","1,698,864",-44.30%
2,2019,"13,581","11,240","23,989","35,491","167,403","544,088","879,711","771,874","488,909","78,408","20,008","15,137","3,049,839",2.90%
3,2018,"12,222","11,847","21,758","28,404","195,116","556,304","905,959","667,688","434,600","91,973","20,657","18,781","2,965,309",-10.30%
4,2017,"14,690","13,802","19,336","38,323","177,787","620,962","1,009,665","908,479","389,137","84,469","15,594","13,268","3,305,512",12.20%
